In [1]:
#https://github.com/silpara/simulators/blob/master/compartmental_models/SEIR%20Simulator%20in%20Python.ipynb
import os
import sys
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.integrate import odeint
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = "notebook"
%matplotlib inline
plt.style.use('ggplot')

# Jupyter Specifics
from IPython.display import HTML
from ipywidgets.widgets import interact, IntSlider, FloatSlider, Layout
style = {'description_width': '100px'}
slider_layout = Layout(width='99%')

In [2]:
### DESCRIPCIÓN DEL MODELO ###

S_0  = 100000       # Susceptibles al día 1
E_0  = 0            # Expuestos al día 1
Y_0  = 1            # Sintomáticos al día 1
A_0  = 1            # Asintomáticos al día 1
R_0  = 0            # Recuperados al día 1
Q1_0 = 0            # Cuarentena al día 1 (asintomáticos)
Q2_0 = 0            # Cuarentena al día 1 (infectados)
Q3_0 = 0            # Cuarentena al día 1 (susceptibles)
phi = 0.55          # Probabilidad de transmisión relativa
alpha = 0.3         # Probabilidad de volverse asintomático tras la infección
gamma = 1/5.1       # (Tasa de incubación), ó (1/gamma) como período de incubación

###############################################################################

epsilon = 0         # Tasa de mortalidad
Gamma = 0           # Tasa de natalidad
lambda_ay  = 0.1     # Proporción de la población que pasa de ser asintomática a infectada
lambda_rs  = 0.1     # Proporción de la población que pasa de ser recuperados a ser susceptibles nuevamente
lambda_sq3 = 0.1     # Proporción de la población que es susceptible y pasa a estar estar en cuarentena
lambda_yq2 = 0.2     # Proporción de la población que es sintomática y pasa a estar en cuarentena
lambda_aq1 = 0.1     # Proporción de la población que es asintomática y pasa a estar en cuarentena
lambda_q1r = 0.1     # Proporción de la población que está en cuarentena y se recupera
lambda_q2r = 0.1     # Proporción de la población que está en cuarentena y se recupera
lambda_q3r = 0.1     # Proporción de la población que está en cuarentena y se recupera
tau = 0.1           # Proporción de la población que respeta la cuarentena 
B_y = 5             # Tasa de contacto efectiva de subpoblaciones sintomáticas a sucseptibles
dias = 30           # días lol
# INPUT = [S, E, Y, A, R, Q]
# parametros = [phi, alpha, gamma, lambda_yr, lambda_ar, B_y]

El modelo:

$\frac{dS}{dt} = \Gamma + \lambda_{RS}R - (B_Y\frac{Y}{N}+B_A\frac{A}{N})(1-\tau)S - \epsilon S - \tau \lambda_{SQ_3}S$

$\frac{dE}{dt} = (B_Y\frac{Y}{N}+B_A\frac{A}{N})(1-\tau) S - \gamma E- \epsilon E$

$\frac{dY}{dt} = \gamma (1-\alpha) E - \epsilon Y - \delta Y - \lambda_{YQ_2}Y + \lambda_{AY}A$

$\frac{dA}{dt} = \gamma \alpha E - \lambda_{AY}A - \lambda_{AQ_1}A - \epsilon A$

$\frac{dR}{dt} = \lambda_{Q_1R}Q_1 +\lambda_{Q_2R}Q_2 + \lambda{Q_3R}Q_3-\epsilon R-\lambda_{RS}R$

$\frac{dQ_1}{dt} = \lambda_{AQ_1}A-\lambda_{Q_1R}Q_1-\epsilon Q_1$

$\frac{dQ_2}{dt} = \lambda_{YQ_2}Y-\lambda_{Q_2R}Q_2-\epsilon Q_2$

$\frac{dQ_3}{dt} = \tau \lambda_{SQ_3}s-\lambda_{Q_3R}Q_3-\epsilon Q_3$

In [3]:
### DEFINICIÓN DEL MODELO ###

def diff_eqs(INP, t, phi, alpha, gamma, epsilon, Gamma, lambda_ay, lambda_rs, lambda_sq3, lambda_yq2, lambda_aq1, lambda_q1r, lambda_q2r, lambda_q3r, tau, B_y):
    """
    Sistema de ecuaciones diferenciales.
    """
    S, E, Y, A, R, Q1, Q2, Q3 = INP
    N = S + E + Y + A + R + Q1 + Q2 + Q3
    
    B_a = B_y * phi                # Tasa de contacto efectivo de subpoblación asintomática a susceptible
    delta = 0.032 * (1 - alpha)    # Tasa de mortalidad inducida por la enfermedad
    
    # Planteamiento matemático del modelo
    
    #dSdt = Gamma + lambda_rs * R - (B_y * (Y/N) + B_a * (A/N)) * (1 - tau) * S - epsilon * S - tau * lambda_sq * S
    #dEdt = (B_y * (Y/N) + B_a * (A/N)) * (1 - tau) * S - gamma * alpha * E - gamma * (1 - alpha) * E - epsilon * E
    #dYdt = gamma * (1 - alpha) * E - epsilon * Y - delta * Y - lambda_yq * Y + lambda_ay * A
    #dAdt = gamma * alpha * E - lambda_ay * A - lambda_aq * A - epsilon * A
    #dRdt = lambda_qr * Q - lambda_rs * R - epsilon * R
    #dQdt = tau * lambda_sq * S + lambda_yq * Y + lambda_aq * A - lambda_qr * Q - epsilon * Q
    #return [dSdt, dEdt, dYdt, dAdt, dRdt, dQdt]
    
    dSdt = Gamma + lambda_rs * R - (B_y * (Y/N) + B_a * (A/N)) * (1 - tau) * S - epsilon * S - tau * lambda_sq3 * S
    dEdt = (B_y * (Y/N) + B_a * (A/N)) * (1 - tau) * S - gamma * E - epsilon * E
    dYdt = gamma * (1 - alpha) * E - epsilon * Y - delta * Y - lambda_yq2 * Y + lambda_ay * A
    dAdt = gamma * alpha * E - lambda_ay * A - lambda_aq1 * A - epsilon * A
    dRdt = lambda_q1r * Q1 + lambda_q2r * Q2 + lambda_q3r * Q3 - lambda_rs * R - epsilon * R
    dQ1dt = lambda_aq1 * A - lambda_q1r * Q1 - epsilon * Q1
    dQ2dt = lambda_yq2 * Y - lambda_q2r * Q2 - epsilon * Q2
    dQ3dt = tau * lambda_sq3 - lambda_q3r * Q3 - epsilon * Q3
    return [dSdt, dEdt, dYdt, dAdt, dRdt, dQ1dt, dQ2dt, dQ3dt]

    #dSdt = Gamma - (B_y * (Y/N) + B_a * (A/N)) * (1 - tau) * S - epsilon * S - tau * lambda_sq * S
    #dEdt = (B_y * (Y/N) + B_a * (A/N)) * (1 - tau) * S - gamma * alpha * E - gamma * (1 - alpha) * E - epsilon * E
    #dYdt = gamma * (1 - alpha) * E - epsilon * Y - delta * Y - lambda_yq * Y + lambda_ay * A
    #dAdt = gamma * alpha * E - lambda_ay * A - lambda_aq * A - epsilon * A
    #dRdt = lambda_qr * Q - epsilon * R
    #dQdt = tau * lambda_sq * S + lambda_yq * Y + lambda_aq * A - lambda_qr * Q - epsilon * Q
    #return [dSdt, dEdt, dYdt, dAdt, dRdt, dQdt]

In [4]:
### SOLUCIÓN DEL MODELO ###

def ode_solver(t, initial_conditions, params):
    S, E, Y, A, R, Q1,Q2, Q3 = initial_conditions
    phi, alpha, gamma, epsilon, Gamma, lambda_ay, lambda_rs, lambda_sq3, lambda_yq2, lambda_aq1, lambda_q1r, lambda_q2r, lambda_q3r, tau, B_y = params
    res = odeint(diff_eqs, [S, E, Y, A, R, Q1, Q2, Q3], t, args=(phi, alpha, gamma, epsilon, Gamma, lambda_ay, lambda_rs, lambda_sq3, lambda_yq2, lambda_aq1, lambda_q1r, lambda_q2r, lambda_q3r, tau, B_y))
    return res

In [5]:
def main(S_0, E_0, Y_0, A_0, R_0, Q1_0, Q2_0, Q3_0, phi, alpha, gamma, epsilon, Gamma, lambda_ay, lambda_rs, lambda_sq3, lambda_yq2, lambda_aq1, lambda_q1r, lambda_q2r, lambda_q3r, tau, B_y, dias):
    initial_conditions = [S_0, E_0, Y_0, A_0, R_0, Q1_0, Q2_0, Q3_0]
    params = (phi, alpha, gamma, epsilon, Gamma, lambda_ay, lambda_rs, lambda_sq3, lambda_yq2, lambda_aq1, lambda_q1r, lambda_q2r, lambda_q3r, tau, B_y)
    periodo = np.arange(0, dias, 1)
    sol = ode_solver(periodo, initial_conditions, params)
    S, E, Y, A, R, Q1, Q2, Q3 = sol[:, 0], sol[:, 1], sol[:, 2], sol[:, 3], sol[:, 4], sol[:, 5], sol[:, 6], sol[:, 7]
    
    # Create traces
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=periodo, y=S, mode='lines+markers', name='Suceptibles'))
    fig.add_trace(go.Scatter(x=periodo, y=E, mode='lines+markers', name='Expuestos'))
    fig.add_trace(go.Scatter(x=periodo, y=Y, mode='lines+markers', name='Sintomaticos'))
    fig.add_trace(go.Scatter(x=periodo, y=A, mode='lines+markers', name='Asintomaticos'))
    fig.add_trace(go.Scatter(x=periodo, y=R, mode='lines+markers', name='Recuperados'))
    fig.add_trace(go.Scatter(x=periodo, y=Q1, mode='lines+markers', name='Cuarentena (asintomáticos)'))
    fig.add_trace(go.Scatter(x=periodo, y=Q2, mode='lines+markers', name='Cuarentena (infectados)'))
    fig.add_trace(go.Scatter(x=periodo, y=Q3, mode='lines+markers', name='Cuarentena (susceptibles)'))
    
    if dias <= 30:
        step = 1
    elif dias <= 90:
        step = 7
    else:
        step = 30
    
    # Edit the layout
    fig.update_layout(title='Simulación de un modelo de Covid-19 con cuarentena',
                       xaxis_title='Día',
                       yaxis_title='Número de personas',
                       title_x=0.5,
                      width=900, height=400
                     )
    fig.update_xaxes(tickangle=-90, tickformat = None, tickmode='array', tickvals=np.arange(0, dias + 1, step))
    if not os.path.exists("images"):
        os.mkdir("images")
    fig.write_image("images/covid_simulation.png")
    fig.show()

In [6]:
interact(main,
         S_0 = IntSlider(min=0, max=100000, step=100, value=S_0, description='Suceptibles', style=style, layout=slider_layout),
         E_0 = IntSlider(min=0, max=100000, step=100, value=E_0, description='Expuestos', style=style, layout=slider_layout),
         Y_0 = IntSlider(min=0, max=100000, step=100, value=Y_0, description='Sintomáticos', style=style, layout=slider_layout),
         A_0 = IntSlider(min=0, max=100000, step=100, value=A_0, description='Asintomaticos', style=style, layout=slider_layout),
         R_0 = IntSlider(min=0, max=100000, step=100, value=R_0, description='Recuperados', style=style, layout=slider_layout),
         Q1_0 = IntSlider(min=0, max=100000, step=100, value=Q1_0, description='Cuarentena (asintomáticos)', style=style, layout=slider_layout),
         Q2_0 = IntSlider(min=0, max=100000, step=100, value=Q2_0, description='Cuarentena (infectados)', style=style, layout=slider_layout),
         Q3_0 = IntSlider(min=0, max=100000, step=100, value=Q3_0, description='Cuarentena (susceptibles)', style=style, layout=slider_layout),
         phi = FloatSlider(min=0.49, max=0.63, step=0.01, value=phi, description='phi', style=style, layout=slider_layout),
         alpha = FloatSlider(min=0, max=1, step=0.01, value=alpha, description='alpha', style=style, layout=slider_layout),
         gamma = FloatSlider(min=0.17, max=0.22, step=0.01, value=gamma, description='gamma', style=style, layout=slider_layout),
         epsilon = FloatSlider(min=0, max=1, step=0.01, value=epsilon, description='epsilon', style=style, layout=slider_layout),
         Gamma = IntSlider(min=0, max=6000, step=1, value=Gamma, description='Gamma', style=style, layout=slider_layout),
         lambda_ay = FloatSlider(min=0, max=1, step=0.01, value=lambda_ay, description='lambda_ay', style=style, layout=slider_layout),
         lambda_rs = FloatSlider(min=0, max=1, step=0.01, value=lambda_rs, description='lambda_rs', style=style, layout=slider_layout),
         lambda_sq3 = FloatSlider(min=0, max=1, step=0.01, value=lambda_sq3, description='lambda_sq3', style=style, layout=slider_layout),
         lambda_yq2 = FloatSlider(min=0, max=1, step=0.01, value=lambda_yq2, description='lambda_yq2', style=style, layout=slider_layout),
         lambda_aq1 = FloatSlider(min=0, max=1, step=0.01, value=lambda_aq1, description='lambda_aq1', style=style, layout=slider_layout),
         lambda_q1r = FloatSlider(min=0, max=1, step=0.01, value=lambda_q1r, description='lambda_q1r', style=style, layout=slider_layout),
         lambda_q2r = FloatSlider(min=0, max=1, step=0.01, value=lambda_q2r, description='lambda_q2r', style=style, layout=slider_layout),
         lambda_q3r = FloatSlider(min=0, max=1, step=0.01, value=lambda_q3r, description='lambda_q3r', style=style, layout=slider_layout),
         tau = FloatSlider(min=0, max=1, step=0.01, value=tau, description='tau', style=style, layout=slider_layout),
         B_y = FloatSlider(min=0, max=20, step=0.1, value=B_y, description='B_y', style=style, layout=slider_layout),
         dias = IntSlider(min=1, max=300, step=7, value=dias, description='Días', style=style, layout=slider_layout)
        );

interactive(children=(IntSlider(value=100000, description='Suceptibles', layout=Layout(width='99%'), max=10000…